In [1]:

!pip uninstall -y transformers accelerate peft gliner numpy

!pip install spacy "transformers==4.57.6" "accelerate==1.12.0" "peft==0.18.1" "gliner==0.2.24" "numpy==2.0.2"


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 112.0 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import transformers
import accelerate
import gliner
import torch
import spacy
import numpy

print(f"Transformers: {transformers.__version__}")
print(f"Accelerate: {accelerate.__version__}")
print(f"GLiNER: {gliner.__version__}")
print(f"Torch: {torch.__version__}")
print(f"Numpy: {numpy.__version__}")

2026-01-29 08:54:31.408588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769676871.610146      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769676871.669924      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769676872.183899      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769676872.183936      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769676872.183939      55 computation_placer.cc:177] computation placer alr

Transformers: 4.57.6
Accelerate: 1.12.0
GLiNER: 0.2.24
Torch: 2.8.0+cu126
Numpy: 2.0.2


In [3]:
import json
import random
import os
import torch
import spacy
from gliner import GLiNER

os.environ["TOKENIZERS_PARALLELISM"] = "false"

try:
    nlp = spacy.load("tr_core_news_tr")
except:
    nlp = spacy.blank("tr")

with open("/kaggle/input/dataset/supplements_spans_ents (1).json", "r", encoding="utf-8") as f:
    raw_user_data = json.load(f)

def merge_labels(data):
    mapping = {
        "MARKA": "URUN_MARKA", "MARKA_DİĞER": "URUN_MARKA", "ÜRÜN_DİĞER": "URUN_MARKA",
        "İÇERİK": "BILESEN", "BİYOMOLEKÜL": "BILESEN",
        "HASTALIK": "SAGLIK_DURUMU", "SAĞLIK_ŞİKAYETLERİ": "SAGLIK_DURUMU",
        "ETKİ": "ETKI_DENEYIM", "YAN_ETKİ": "ETKI_DENEYIM",
        "KULLANICI": "KULLANICI_BILGISI", "KULLANICI_DEMOGRAFİSİ": "KULLANICI_BILGISI",
        "TAVSİYE_EDEN": "TAVSİYE_EDEN",
        "DOZ":"DOZ",
        "TAT_KOKU":"TAT_KOKU"
    }
    
    for item in data:
        for ent in item["entities"]:
            if ent["label"] in mapping:
                ent["label"] = mapping[ent["label"]]
    return data

raw_user_data = merge_labels(raw_user_data)

print("Sade Label Listesi:", list(set(ent['label'] for item in raw_user_data for ent in item['entities'])))

def prepare_data(raw_data):
    final_data = []
    for item in raw_data:
        text = item["text"]
        doc = nlp(text)
        converted_entities = []
        for ent in item["entities"]:
            span = doc.char_span(ent["start"], ent["end"], label=ent["label"], alignment_mode="contract")
            if span:
                converted_entities.append([span.start, span.end, span.label_])
        
        final_data.append({
            "tokenized_text": [t.text for t in doc],
            "ner": converted_entities
        })
    return final_data


dataset = prepare_data(raw_user_data)



Sade Label Listesi: ['ETKI_DENEYIM', 'TAVSİYE_EDEN', 'TAT_KOKU', 'BILESEN', 'URUN_MARKA', 'DOZ', 'SAGLIK_DURUMU', 'KULLANICI_BILGISI']


In [4]:
dataset[1]

{'tokenized_text': ['Diğer',
  'demir',
  'ilaçlarından',
  'farklı',
  'olarak',
  'mideyi',
  'rahatsız',
  'etmiyor',
  '.',
  'Saç',
  'dökülmemi',
  'engelliyor',
  'yıllardır',
  'kullanıyorum',
  '.'],
 'ner': [[1, 3, 'URUN_MARKA'],
  [5, 8, 'ETKI_DENEYIM'],
  [9, 11, 'SAGLIK_DURUMU'],
  [9, 12, 'ETKI_DENEYIM']]}

In [5]:
random.shuffle(dataset)
split_point = int(len(dataset) * 0.8)
train_dataset = dataset[:split_point]
test_val = dataset[split_point:]

split_point_val_test = int(len(test_val) * 0.5)
eval_dataset = test_val[:split_point_val_test]
test_dataset = test_val[split_point_val_test:]

print(f"Eğitim seti:   {len(train_dataset)} ")
print(f"Doğrulama seti: {len(eval_dataset)} ")
print(f"Final Test seti: {len(test_dataset)} ")


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
model.to(device)


model.train_model(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    output_dir="gliner_finetuned_optimized",

    load_best_model_at_end=True,     
    metric_for_best_model="eval_loss", 
    greater_is_better=False,           
    save_strategy="epoch",            
    eval_strategy="epoch",             
    save_total_limit=1,                
    
    learning_rate=5e-6,
    others_lr=5e-5,                   
    weight_decay=0.01,
    others_weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    
    num_train_epochs=10,                
    max_steps=-1,                      
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    focal_loss_alpha=0.75,
    focal_loss_gamma=2,
    
    dataloader_num_workers=0,
    report_to="none",
    bf16=False, 
    fp16=False                         
)

output_dir="/kaggle/working/best_model_10epoch_final"
model.save_pretrained(output_dir)

Eğitim seti:   1977 
Doğrulama seti: 247 
Final Test seti: 248 
cuda:0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gliner/model.py:1093: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1, 'pad_token_id': 0}.
Aski

Epoch,Training Loss,Validation Loss
1,10.292300,340.748077
2,8.341300,219.241592
3,6.623800,296.067444
4,5.374400,196.222900
5,5.262900,181.849319
6,5.591100,152.121292
7,5.016700,121.592613
8,5.568900,149.095306
9,4.689500,126.451416
10,4.590900,120.742393


There were missing keys in the checkpoint model loaded: ['model.token_rep_layer.bert_layer.model.embeddings.word_embeddings.weight', 'model.token_rep_layer.bert_layer.model.embeddings.LayerNorm.weight', 'model.token_rep_layer.bert_layer.model.embeddings.LayerNorm.bias', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.self.query_proj.weight', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.self.query_proj.bias', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.self.key_proj.weight', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.self.key_proj.bias', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.self.value_proj.weight', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.self.value_proj.bias', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.output.dense.weight', 'model.token_rep_layer.bert_layer.model.encoder.layer.0.attention.output.dense.bias', 'model.token_rep_la

In [6]:
all_labels = set()

for item in raw_user_data:
    for ent in item["entities"]:
        all_labels.add(ent["label"])

labels_list = list(all_labels)

print(f"tüm etiketler ({len(labels_list)})")
print(labels_list)

tüm etiketler (8)
['ETKI_DENEYIM', 'TAVSİYE_EDEN', 'TAT_KOKU', 'BILESEN', 'URUN_MARKA', 'DOZ', 'SAGLIK_DURUMU', 'KULLANICI_BILGISI']


In [7]:

test_dataset[1]

{'tokenized_text': ['Benim',
  'iştahımı',
  'azalttı',
  ',',
  'tadı',
  'yok',
  'bence',
  '.',
  'Ancak',
  'eminim',
  'bende',
  'baş',
  'ağrısı',
  'yapıyor',
  '.',
  'Bu',
  'sebeple',
  'devam',
  'edip',
  'etmemeye',
  'karar',
  'veremedim',
  'henüz',
  '.'],
 'ner': [[4, 6, 'TAT_KOKU'], [11, 13, 'ETKI_DENEYIM'], [1, 3, 'ETKI_DENEYIM']]}

In [8]:
import random

model_gliner = GLiNER.from_pretrained(output_dir)

samples = random.sample(list(test_dataset), 10)

print("-" * 70)
print(f"{'TEST':^70}")
print("-" * 70)

for i, sample in enumerate(samples, 1):

    text = " ".join(sample["tokenized_text"])
    
    print(f"Test {i}: {text}")
    
    print("\n[ORİJİNAL VARLIKLAR ]")
    original_entities = []
    for start, end, label in sample['ner']:
        extracted_words = sample['tokenized_text'][start:end]
        original_phrase = " ".join(extracted_words)
        original_entities.append(f"{original_phrase} => {label}")
    
    if not original_entities:
        print("   Orijinal veride etiket bulunmuyor.")
    else:
        for ent in original_entities:
            print(f"    {ent}")

    print("\n[MODEL TAHMİNİ]")
    entities = model_gliner.predict_entities(text, labels_list, threshold=0.4)
    
    if not entities:
        print("    Herhangi bir entity tespit edilemedi.")
    else:
        for ent in entities:
            print(f"     {ent['text']} => {ent['label']} (Güven: {ent['score']:.2f})")

    print("\n" + "-" * 70)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


----------------------------------------------------------------------
                                 TEST                                 
----------------------------------------------------------------------
Test 1: İki aldır kullanıyorum fakat saç dökülmem inanılmaz arttı nedeni nedir bilmiyorum şimdilik hiç faydasını göremedim fakat kullanmayı da bırakmadım devam ediyorum bakalım ne olacak

[ORİJİNAL VARLIKLAR ]
    İki aldır kullanıyorum fakat saç dökülmem inanılmaz arttı => ETKI_DENEYIM
    saç dökülmem => SAGLIK_DURUMU

[MODEL TAHMİNİ]
     İki aldır kullanıyorum => DOZ (Güven: 0.79)
     saç dökülmem inanılmaz => SAGLIK_DURUMU (Güven: 0.85)
     şimdilik hiç faydasını göremedim fakat => ETKI_DENEYIM (Güven: 0.41)

----------------------------------------------------------------------
Test 2: ürünü geçen yıl ramazan ayında da kullanmıştım ( düzenli olarak değil tabi kaçırdığım unuttuğum zamanlar oldu)aslında tek öğün beslenme için takviye olarak tüketmiştim tabi yanında b12 v